In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df = application_df.drop(['EIN','NAME','SPECIAL_CONSIDERATIONS'],axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1


In [2]:
# Determine which values to replace if counts are less than ...?
application_count = application_df['APPLICATION_TYPE'].value_counts()
replace_application = list(application_count[application_count < 200].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [3]:
# Determine which values to replace if counts are less than ..?
classification_count = application_df['CLASSIFICATION'].value_counts()
replace_class = list(classification_count[classification_count < 1700].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [4]:
affiliation_count = application_df['AFFILIATION'].value_counts()
affiliation_count

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ..?
affiliation_count = application_df['AFFILIATION'].value_counts()
replace_class = list(affiliation_count[affiliation_count < 100].index)

# Replace in dataframe
for cls in replace_class:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.AFFILIATION.value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [6]:
income_count = application_df.INCOME_AMT.value_counts()
replace_class = list(income_count[income_count < 250].index)

# Replace in dataframe
for cls in replace_class:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(cls,"5M+")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
5M+                564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [7]:
application_df.USE_CASE.value_counts()

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [8]:
use_count = application_df.USE_CASE.value_counts()
replace_class = list(use_count[use_count < 600].index)

# Replace in dataframe
for cls in replace_class:
    application_df.USE_CASE = application_df.USE_CASE.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.USE_CASE.value_counts()

Preservation    28095
ProductDev       5671
Other             533
Name: USE_CASE, dtype: int64

In [9]:
# Generate our categorical variable lists
application_cat=['APPLICATION_TYPE','CLASSIFICATION', 'USE_CASE','ORGANIZATION','AFFILIATION',
                 'INCOME_AMT']

In [10]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat].values.reshape(-6,6)))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,CLASSIFICATION_C1000,...,AFFILIATION_CompanySponsored,AFFILIATION_Independent,AFFILIATION_Other,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [11]:
# Merge one-hot encoded features and drop the originals
application_encoded_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,axis=1)
application_encoded_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,AFFILIATION_CompanySponsored,AFFILIATION_Independent,AFFILIATION_Other,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [12]:
# Split our preprocessed data into our features and target arrays
y = application_encoded_df['IS_SUCCESSFUL'].values
X = application_encoded_df.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2,activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3,activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 280       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 2)                 12        
                                                                 
 dense_3 (Dense)             (None, 1)                 3         
                                                                 
Total params: 340
Trainable params: 340
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
fit_model = nn.fit(X_train,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 4s 3ms/step - loss: 0.6913 - accuracy: 0.5346
Epoch 2/50
804/804 [==============================] - 5s 6ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 3/50
804/804 [==============================] - 3s 4ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 4/50
804/804 [==============================] - 3s 4ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 5/50
804/804 [==============================] - 3s 4ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.5347

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7068 - accuracy: 0.4771 - 455ms/epoch - 2ms/step
Loss: 0.7067877054214478, Accuracy: 0.47708454728126526


## ATTEMPT 2

In [18]:
application_df = pd.read_csv("charity_data.csv")
application_df = application_df.drop(['EIN','NAME','SPECIAL_CONSIDERATIONS','STATUS','ORGANIZATION','USE_CASE'],axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,0,5000,1
1,T3,Independent,C2000,1-9999,108590,1
2,T5,CompanySponsored,C3000,0,5000,0
3,T3,CompanySponsored,C2000,10000-24999,6692,1
4,T3,Independent,C1000,100000-499999,142590,1


In [19]:
# Determine which values to replace if counts are less than ...?
application_count = application_df['APPLICATION_TYPE'].value_counts()
replace_application = list(application_count[application_count < 200].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [20]:
# Determine which values to replace if counts are less than ..?
affiliation_count = application_df['AFFILIATION'].value_counts()
replace_class = list(affiliation_count[affiliation_count < 100].index)

# Replace in dataframe
for cls in replace_class:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.AFFILIATION.value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [21]:
# Determine which values to replace if counts are less than ..?
classification_count = application_df['CLASSIFICATION'].value_counts()
replace_class = list(classification_count[classification_count < 1700].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [22]:
income_count = application_df.INCOME_AMT.value_counts()
replace_class = list(income_count[income_count < 250].index)

# Replace in dataframe
for cls in replace_class:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(cls,"5M+")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
5M+                564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [23]:
# Generate our categorical variable lists
application_cat=['APPLICATION_TYPE','CLASSIFICATION','AFFILIATION','INCOME_AMT']

In [24]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat].values.reshape(-4,4)))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,CLASSIFICATION_C1000,...,AFFILIATION_CompanySponsored,AFFILIATION_Independent,AFFILIATION_Other,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [25]:
# Merge one-hot encoded features and drop the originals
application_encoded_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,axis=1)
application_encoded_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,AFFILIATION_CompanySponsored,AFFILIATION_Independent,AFFILIATION_Other,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [26]:
# Split our preprocessed data into our features and target arrays
y = application_encoded_df['IS_SUCCESSFUL'].values
X = application_encoded_df.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

In [27]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='swish'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2,activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3,activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 8)                 216       
                                                                 
 dense_5 (Dense)             (None, 5)                 45        
                                                                 
 dense_6 (Dense)             (None, 2)                 12        
                                                                 
 dense_7 (Dense)             (None, 1)                 3         
                                                                 
Total params: 276
Trainable params: 276
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
fit_model = nn.fit(X_train,y_train,epochs=25)

Epoch 1/25
804/804 [==============================] - 2s 2ms/step - loss: 0.6914 - accuracy: 0.5342
Epoch 2/25
804/804 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 3/25
804/804 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 4/25
804/804 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 5/25
804/804 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 6/25
804/804 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 7/25
804/804 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 8/25
804/804 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 9/25
804/804 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.5347
Epoch 10/25
804/804 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.5347

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7033 - accuracy: 0.4736 - 428ms/epoch - 2ms/step
Loss: 0.703328013420105, Accuracy: 0.47358599305152893


## ATTEMPT 3

In [32]:
application_df = pd.read_csv("charity_data.csv")
application_df = application_df.drop(['EIN','NAME','SPECIAL_CONSIDERATIONS','STATUS','ORGANIZATION','USE_CASE','AFFILIATION','APPLICATION_TYPE','CLASSIFICATION','ASK_AMT'],axis=1)
application_df.head()

,INCOME_AMT,IS_SUCCESSFUL
0,0,1
1,1-9999,1
2,0,0
3,10000-24999,1
4,100000-499999,1


In [33]:
income_count = application_df.INCOME_AMT.value_counts()
replace_class = list(income_count[income_count < 250].index)

# Replace in dataframe
for cls in replace_class:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(cls,"5M+")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
5M+                564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [34]:
# Generate our categorical variable lists
application_cat=['INCOME_AMT']

In [35]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat].values.reshape(-1,1)))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [36]:
#Merge one-hot encoded features and drop the originals
application_encoded_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,axis=1)
application_encoded_df.head()

,IS_SUCCESSFUL,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+
0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [37]:
# Split our preprocessed data into our features and target arrays
y = application_encoded_df['IS_SUCCESSFUL'].values
X = application_encoded_df.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

In [38]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [39]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='swish'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2,activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3,activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 20)                160       
                                                                 
 dense_9 (Dense)             (None, 10)                210       
                                                                 
 dense_10 (Dense)            (None, 5)                 55        
                                                                 
 dense_11 (Dense)            (None, 1)                 6         
                                                                 
Total params: 431
Trainable params: 431
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [41]:
cp_callback = ModelCheckpoint(
    filepath="AlphabetSoupCharity_Optimization.h5",
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

fit_model = nn.fit(X_train,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 3s 2ms/step - loss: 0.6891 - accuracy: 0.5329
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.6873 - accuracy: 0.5361
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.6870 - accuracy: 0.5377
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.6864 - accuracy: 0.5389
Epoch 5/50
800/804 [============================>.] - ETA: 0s - loss: 0.6862 - accuracy: 0.5375
Epoch 5: saving model to AlphabetSoupCharity_Optimization.h5
804/804 [==============================] - 1s 2ms/step - loss: 0.6862 - accuracy: 0.5376
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6863 - accuracy: 0.5389
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.6863 - accuracy: 0.5389
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.6862 - accuracy: 0.5389
Epoch 9/50
804/804 [==============================] - 

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7506 - accuracy: 0.5324 - 433ms/epoch - 2ms/step
Loss: 0.7505708336830139, Accuracy: 0.5323615074157715
